<a href="https://www.kaggle.com/code/amirmotefaker/coronavirus-pandemic-covid-19-visualization?scriptVersionId=115478996" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Coronavirus Pandemic (COVID-19)

- Coronavirus disease (COVID-19) is an infectious disease caused by the SARS-CoV-2 virus.
- Most people who fall sick with COVID-19 will experience mild to moderate symptoms and recover without special treatment. However, some will become seriously ill and require medical attention.

Learn more about Coronavirus disease (COVID-19) pandemic: [WHO](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)

COVID Live - Coronavirus Statistics: [Worldometers](https://www.worldometers.info/coronavirus/)

COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University: [Data](https://github.com/CSSEGISandData/COVID-19)


# Import Libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time

from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

import datetime
import operator  # exports a set of efficient functions corresponding to the intrinsic operators of Python
plt.style.use('seaborn-poster')

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


## Import the data (make sure you update this on a daily basis)

In [2]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/09-09-2022.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/09-09-2022.csv')

In [3]:
latest_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-09-10 04:21:10,33.93911,67.709953,195471,7786,NaN,NaN,Afghanistan,502.130312,3.983200
1,NaN,NaN,NaN,Albania,2022-09-10 04:21:10,41.15330,20.168300,330842,3584,NaN,NaN,Albania,11496.351380,1.083297
2,NaN,NaN,NaN,Algeria,2022-09-10 04:21:10,28.03390,1.659600,270522,6879,NaN,NaN,Algeria,616.911210,2.542862
3,NaN,NaN,NaN,Andorra,2022-09-10 04:21:10,42.50630,1.521800,46113,155,NaN,NaN,Andorra,59681.615220,0.336131
4,NaN,NaN,NaN,Angola,2022-09-10 04:21:10,-11.20270,17.873900,103131,1917,NaN,NaN,Angola,313.789810,1.858801


In [4]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/25/22,12/26/22,12/27/22,12/28/22,12/29/22,12/30/22,12/31/22,1/1/23,1/2/23,1/3/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,207399,207438,207460,207493,207511,207550,207559,207616,207627,207654
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333749,333751,333751,333776,333776,333806,333806,333811,333812,333812
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271198,271198,271202,271208,271217,271223,271228,271229,271229,271230
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47686,47686,47686,47751,47751,47751,47751,47751,47751,47751
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,104973,104973,105095,105095,105095,105095,105095,105095,105095,105095


In [5]:
us_medical_data.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,Date,People_Tested,Mortality_Rate
0,Alabama,US,2022-09-10 04:33:21,32.3182,-86.9023,1504180,20239,NaN,NaN,1.0,...,NaN,NaN,1.345517,84000001.0,USA,NaN,NaN,2022-09-09,NaN,NaN
1,Alaska,US,2022-09-10 04:33:21,61.3707,-152.4044,295596,1338,NaN,NaN,2.0,...,NaN,NaN,0.452645,84000002.0,USA,NaN,NaN,2022-09-09,NaN,NaN
2,American Samoa,US,2022-09-10 04:33:21,-14.2710,-170.1320,8217,34,NaN,NaN,60.0,...,NaN,NaN,0.413776,16.0,ASM,NaN,NaN,2022-09-09,NaN,NaN
3,Arizona,US,2022-09-10 04:33:21,33.7298,-111.4312,2258040,31162,NaN,NaN,4.0,...,20720327.0,NaN,1.380046,84000004.0,USA,284670.045559,NaN,2022-09-09,NaN,NaN
4,Arkansas,US,2022-09-10 04:33:21,34.9697,-92.3731,940689,11953,NaN,NaN,5.0,...,5477770.0,NaN,1.270664,84000005.0,USA,181515.101710,NaN,2022-09-09,NaN,NaN


In [6]:
confirmed_cols = confirmed_df.keys()
deaths_cols = deaths_df.keys()

## Get all the dates for the ongoing coronavirus pandemic

In [7]:
confirmed = confirmed_df.loc[:, confirmed_cols[4]:]
deaths = deaths_df.loc[:, deaths_cols[4]:]

In [8]:
confirmed.keys()

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '12/25/22', '12/26/22', '12/27/22', '12/28/22', '12/29/22', '12/30/22',
       '12/31/22', '1/1/23', '1/2/23', '1/3/23'],
      dtype='object', length=1078)

In [9]:
num_dates = len(confirmed.keys())
ck = confirmed.keys()
dk = deaths.keys()

world_cases = []
total_deaths = [] 
mortality_rate = []


for i in range(num_dates):
    confirmed_sum = confirmed[ck[i]].sum()
    death_sum = deaths[dk[i]].sum()
    
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)